In [1]:
import requests
import pandas as pd
import time

In [28]:
# the components of the API call, which make up the "request" variable
url = 'https://api.legiscan.com/?key='
key = 'f48c01578e3d8481ec9038b9df67830d'
page = 1
request = requests.get(url + key + '&op=getSearch&state=ALL&query=marijuana' + '&page=' + str(page))

# to print out the full URL, and we can paste in the browser to get 
# an interactive look at the raw results
print(url + key + '&op=getSearch&state=ALL&query=marijuana' + '&page=' + str(page))

https://api.legiscan.com/?key=f48c01578e3d8481ec9038b9df67830d&op=getSearch&state=ALL&query=marijuana&page=1


In [29]:
# get the page_total and the count from the request summary
page_total = request.json()['searchresult']['summary']['page_total']
count = request.json()['searchresult']['summary']['count']
print('Page total: ' + str(page_total) + '\n' + 'Total results: ' + str(count))

Page total: 23
Total results: 1128


In [30]:
# request the additional pages of the query by adding 1 to the 'page' 
# parameter until it reaches the page_total. Store each page of  
# requests in a list. Wait 3 seconds between each request to avoid 
# overloading the API

pages = []
for i in range(page_total):
    page = i + 1
    request = requests.get(url + key + '&op=getSearch&state=ALL&query=marijuana' + '&page=' + str(page))
    time.sleep(3)
    pages.append(request.json())

In [31]:
# for each page of the request, parse the results and add them to a 
# dataframe. each page is a json file with individual results labeled
# '0' through '49' and # 'summary' nested under searchresult. Ignore 
# the summary and use pandas.concat to add each of the results from 
# the request in a dataframe

df = pd.DataFrame()
for page in pages:
    results = page['searchresult']
    # if the page has no results, skip it
    for i in range(50):
        if str(i) in results:
            df = pd.concat([df, pd.DataFrame(results[str(i)], index=[i])])
        else:
            continue

In [33]:
print(df)

    relevance state bill_number  bill_id                       change_hash  \
0         100    CO      HB1061  1810748  a1c18e4309764f2f66611d6a44c33dd3   
1          99    CO       SB076  1824252  253d208f013a36be84633a3f251503f5   
2          99    SC       S0211  1635367  5f57e929eccdf774ca41707c6cc59430   
3          99    CO       SB172  1859475  ee04ab9c894cc20242d6435bdf3de28c   
4          99    AK       HB119  1743726  25dd8b7c73d3a2721f38eaa40d3f6873   
..        ...   ...         ...      ...                               ...   
23         14    WV       SB421  1812739  edd5d084f5d282760e6ffdb9fb6efddc   
24         13    MI      HB4292  1743262  9800622285060da7e2386e9f064a91a8   
25          5    NJ       A4700  1883494  f5124daa49c522c2413e007df43f5020   
26          5    NJ       S2025  1883535  d328c57e3ec173c669a9ef402ca009ab   
27          4    MI      HB4437  1757146  604fc33c04c0b4d89cce5b8bd0d51893   

                                         url  \
0   https://leg

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128 entries, 0 to 27
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   relevance         1128 non-null   int64 
 1   state             1128 non-null   object
 2   bill_number       1128 non-null   object
 3   bill_id           1128 non-null   int64 
 4   change_hash       1128 non-null   object
 5   url               1128 non-null   object
 6   text_url          1128 non-null   object
 7   research_url      1128 non-null   object
 8   last_action_date  1128 non-null   object
 9   last_action       1128 non-null   object
 10  title             1128 non-null   object
dtypes: int64(2), object(9)
memory usage: 105.8+ KB


In [37]:
df.to_csv('legiscan_api_results.csv')